In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests 
from collections import defaultdict
import lxml.html as lh

In [2]:
URL='https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-eff:prev~next=100~300|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=all'
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
url_series=soup.find_all('a',class_='title')
path_dic={}
path=[]
for item in url_series:
    path_dic[item.text]=item.get('href')
    path.append(item.get('href'))
    

In [5]:
print(len(path))

107


In [26]:
def extrac_data(path_dic,base_url):
    dic_df={}
    for car in path_dic.keys():
        page=requests.get(base_url+path_dic[car])
        soup_s = BeautifulSoup(page.content, 'html.parser')
        df=soup_s.find_all('div',class_='data-table')
        data=defaultdict(list)
        for item in df:   
            t=item.text.strip().split('\n')
            for i in range(len(t)):
                if i>0:
                    if t[i]!='':
                        data[t[0]].append(t[i]) 
        dic_df[car]=dict(data)       
    return dic_df    
         
        
    

In [27]:
base_url='https://ev-database.org'
dic_df=extrac_data(path_dic,base_url)

In [47]:
import pandas as pd 
csv=pd.DataFrame()
for item in dic_df.keys():
    for a in dic_df[item].keys():
        if 'Real Energy Consumption' in a:
            for i in range(len(dic_df[item][a])):
                
                if i%2==0:
                    csv.loc[item,dic_df[item][a][i].replace(' *','')]=dic_df[item][a][i+1].replace(' Wh/km','')
            
           
            
        

In [49]:
csv.head(5)

,City - Cold Weather,Highway - Cold Weather,Combined - Cold Weather,City - Mild Weather,Highway - Mild Weather,Combined - Mild Weather
Volkswagen ID.3 Pure,170,237,196,111,180,143
Volkswagen ID.4 1st,203,280,237,139,223,177
Lucid Air,180,232,202,120,177,147
Skoda Enyaq iV 50,186,260,217,125,200,160
Tesla Model 3 Long Range Dual Motor,171,223,193,112,171,139


In [59]:
a={}
for item in csv.columns :
    a[item]=item+ ' Wh/km'
    csv=csv.rename(columns=a)
        

In [60]:
csv

,City - Cold Weather Wh/km,Highway - Cold Weather Wh/km,Combined - Cold Weather Wh/km,City - Mild Weather Wh/km,Highway - Mild Weather Wh/km,Combined - Mild Weather Wh/km
Volkswagen ID.3 Pure,170,237,196,111,180,143
Volkswagen ID.4 1st,203,280,237,139,223,177
Lucid Air,180,232,202,120,177,147
Skoda Enyaq iV 50,186,260,217,125,200,160
Tesla Model 3 Long Range Dual Motor,171,223,193,112,171,139
Honda e,173,238,204,112,184,146
Volkswagen e-Golf,173,237,200,114,188,149
Tesla Model 3 Standard Range Plus,161,216,186,104,164,132
Polestar 2,193,264,223,132,204,165
Audi e-tron GT,204,270,233,140,209,173


In [1]:
csv=csv.rename(columns=dict(a))

In [61]:
csv.to_csv('EV data.csv')